In [16]:
! pip install gensim --quiet

In [17]:
import pandas
import matplotlib.pyplot as plt
import seaborn as sns
import re
import json
%matplotlib inline
import nltk
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix, classification_report, matthews_corrcoef

In [19]:
import numpy as np

## Splitting

In [20]:
import pandas as pd

# Membaca data dari file CSV
df = pd.read_csv("/content/drive/My Drive/RF/data-fix.csv")

# Mengubah nama kolom dari 0 menjadi Comment
df.rename(columns={'0': 'Comment'}, inplace=True)
print(df.head())


                                             Comment sentiment
0  ['dokter', 'industri', 'sabar', 'panjang', 'ma...    netral
1  ['sumpah', 'doa', 'biar', 'dokter', 'boyke', '...   positif
2              ['sehat', 'selalu', 'doker', 'boyke']   positif
3  ['sehat', 'panjang', 'umur', 'selalu', 'drboyk...   positif
4     ['tetap', 'dokter', 'pacar', 'gerbang', 'jah']   positif


In [21]:
label = {
    'positif': 0,
    'netral': 1,
    'negatif': 2
}

In [22]:
df["sentiment"] = df["sentiment"].map(label)

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(df['Comment'],
                                                    df['sentiment'],
                                                    test_size=0.2,
                                                    random_state=15)

In [24]:
X_train

,Comment
209,"['sehat', 'terus', 'pak', 'boy']"
79,"['sehat', 'selalu', 'and', 'panjang', 'umur', 'dokter', 'boyke']"
186,"['pak', 'dokter', 'boyke', 'mod', 'sangat', 'kalau', 'ceritabikin', 'ngkak', 'mululucu', 'sangat']"
61,"['next', 'dokter', 'tirta']"
233,"['betul', 'komunikasi', 'sangat', 'penting', 'dalam', 'keluarga', 'orang', 'tua', 'zaman', 'dulu', 'tahu', 'sendiri', 'jngankan', 'beri', 'didik', 'sex', 'bicara', 'pnjang', 'lebar', 'jarang', 'sekali', 'semua', 'banyak', 'itu', 'kalian', 'rasa', 'tidak', 'sih', 'guys']"
230,"['saman', 'ku', 'beda', 'anak', 'dua', 'cowok', 'anak', 'pertama', 'cewek', 'jadi', 'search', 'cara', 'ny', 'bagaimana', 'temu', 'edukasi', 'ny', 'dari', 'boyke', 'wawalahualam', 'benar', 'dong', 'anak', 'dua', 'lahir', 'cowok']"
279,"['maf', 'dokter', 'teori', 'otak', 'kanan', 'otak', 'kiri', 'bukti', 'buah', 'mitos']"
86,"['dokter', 'boyke', 'dokter', 'djaja', 'kali']"
110,"['dia', 'sunguh', 'dokter', 'satu', 'bangsa', 'boyke', 'mang', 'paling', 'top']"
321,"['dokter', 'satu', 'bangsa']"


In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer= TfidfVectorizer()
tf_x_train = vectorizer.fit_transform(X_train).toarray()
tf_x_test = vectorizer.transform(X_test).toarray()

In [26]:
tf_x_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [27]:
tf_x_test

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [28]:
# Convert to float32
tf_x_train = tf_x_train.astype(np.float32)
tf_x_test = tf_x_test.astype(np.float32)

## CNN

In [29]:
# # #Gunakan Code Untuk Ngecek keseluruhan data, dikarnakan Google Coolabs saya tidak memadai untuk menguji coba semua data

# import numpy as np
# from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
# from sklearn.model_selection import train_test_split
# from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
# import torch

# # Step 3: Define labels (convert to integers if necessary, assuming already done)
# Y_train = np.array(Y_train)
# Y_test = np.array(Y_test)

# # Step 4: Fine-tune BERT for Sequence Classification
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

# # Convert data to PyTorch tensors
# train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True)
# test_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True)

# class Dataset(torch.utils.data.Dataset):
#     def __init__(self, encodings, labels):
#         self.encodings = encodings
#         self.labels = labels

#     def __getitem__(self, idx):
#         item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
#         item['labels'] = torch.tensor(self.labels[idx])
#         return item

#     def __len__(self):
#         return len(self.labels)

# train_dataset = Dataset(train_encodings, Y_train)
# test_dataset = Dataset(test_encodings, Y_test)

# # Training arguments
# training_args = TrainingArguments(
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     num_train_epochs=3,
#     logging_dir='./logs',
#     logging_steps=10,
#     save_steps=500,
#     evaluation_strategy='epoch',
#     output_dir='./results'
# )

# # Trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=test_dataset,
# )

# # Fine-tune BERT
# trainer.train()

# # Step 5: Evaluation
# predictions, labels, _ = trainer.predict(test_dataset)

# # Calculate metrics
# accuracy = accuracy_score(labels, np.argmax(predictions, axis=1))
# recall = recall_score(labels, np.argmax(predictions, axis=1), average='macro')
# precision = precision_score(labels, np.argmax(predictions, axis=1), average='macro')
# f1 = f1_score(labels, np.argmax(predictions, axis=1), average='macro')

# print(f'Accuracy: {accuracy}')
# print(f'Recall: {recall}')
# print(f'Precision: {precision}')
# print(f'F1-score: {f1}')

In [30]:
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
from torch.utils.data import DataLoader, SequentialSampler

# Step 3: Define labels (convert to integers if necessary, assuming already done)
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)

# Step 4: Fine-tune BERT for Sequence Classification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

# Convert data to PyTorch tensors
train_encodings = tokenizer(X_train.tolist(), truncation=True, padding=True)
test_encodings = tokenizer(X_test.tolist(), truncation=True, padding=True)

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = Dataset(train_encodings, Y_train)
test_dataset = Dataset(test_encodings, Y_test)

# Training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=8,  # Reduce batch size if memory is a concern
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=500,
    evaluation_strategy='epoch',
    output_dir='./results'
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

# Fine-tune BERT
trainer.train()

# Step 5: Evaluation with batch processing to reduce memory usage
def evaluate_model(trainer, test_dataset, batch_size=8):
    test_sampler = SequentialSampler(test_dataset)
    test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=batch_size)

    all_predictions = []
    all_labels = []

    for batch in test_dataloader:
        batch = {k: v.to(trainer.args.device) for k, v in batch.items()}

        with torch.no_grad():
            outputs = trainer.model(**batch)

        predictions = np.argmax(outputs.logits.cpu().numpy(), axis=1)
        labels = batch['labels'].cpu().numpy()

        all_predictions.extend(predictions)
        all_labels.extend(labels)

    return np.array(all_predictions), np.array(all_labels)

# Evaluate model
predictions, labels = evaluate_model(trainer, test_dataset)

# Calculate metrics
accuracy = accuracy_score(labels, predictions)
recall = recall_score(labels, predictions, average='macro')
precision = precision_score(labels, predictions, average='macro')
f1 = f1_score(labels, predictions, average='macro')

print(f'Accuracy: {accuracy}')
print(f'Recall: {recall}')
print(f'Precision: {precision}')
print(f'F1-score: {f1}')


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.007000,0.001835
2,0.001400,0.001021


Epoch,Training Loss,Validation Loss
1,0.007000,0.001835
2,0.001400,0.001021
3,0.001100,0.000879


Accuracy: 1.0
Recall: 1.0
Precision: 1.0
F1-score: 1.0
